In [42]:
#coding: utf-8
import requests
from bs4 import BeautifulSoup
import datetime
import re
from konlpy.tag import Twitter
import urllib.request

In [52]:
#크롤링함수 url의 본문 내용 저장
def get_text(URL):
    source_code_from_URL = urllib.request.urlopen(URL)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = ''
    for item in soup.find_all('div', id='articleBodyContents'):
        text = text + str(item.find_all(text=True))
    return text

In [53]:
#클리닝함수 본문 내용 클리닝
def clean_text(text):   
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;▶“’ⓒ:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', cleaned_text)
    return cleaned_text

In [62]:
# text저장 함수
# url을 get_text(url)을 통해 기사의 내용을 가져온후 clean_text 함수를 통해 clean_text로 
# 변경시켜 준후 명사화 시켜서 저장 
def article_text(URL, num, day):
    spliter = Twitter()
    try:
        OUTPUT_FILE_NAME = 'article_dict/'+ day +'/article_'+ day +'_'+ str(num) +'.txt'
        open_output_file = open(OUTPUT_FILE_NAME, 'w')
        result_text = clean_text(get_text(URL))
        article_nouns = spliter.nouns(result_text)
        for n in article_nouns: 
            open_output_file.write(' ' + n)
        open_output_file.close()
        
    finally:
        open_output_file.close()

In [55]:
#db에 저장하는 함수
def db_manager(db_id, url, title, day):
    """
    db에 저장시킬 sql문 작성
    명사화된 title저장
    INSERT INTO db_article VALUES('db_id','url','title','nouns_title','day');
    명사화된 title저장안하기
    INSERT INTO db_article VALUES('db_id','url','title','day');
    """

In [49]:
def main():
    page = 1
    temp_page = 0
    #db에 저장 시킬 id 날짜 별로 구별 필요
    db_id = 0
    first_url = ''
    spliter = Twitter()
    #now는 오늘 날짜를 통해서 전날의 뉴스들을 crawling 해서 저장
    now = datetime.datetime.now()
    yesterday = now - datetime.timedelta(days=1)
    yesterdayDate = yesterday.strftime('%Y%m%d')
    
    #dict에 있는 article_text를 저장하기위해 만든 변수
    num = 0
    
    while 1 == 1:
        url = 'http://news.naver.com/main/list.nhn?sid1=100&listType=title&mid=sec&mode=LSD&date='+str(yesterdayDate)+'&page='+str(page)
        source_code = requests.get(url)
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text,'lxml')
        
        for body in soup.find_all("a", class_="nclicks(fls.list)"):
             
            #db에 저장 시킬 url
            url = body.get('href')
            
            #마지막 페이지에 도달할시 함수 종료
            if first_url == url:
                return
            #db에 저장 시킬 article title
            title = str(body.find_all(text=True))
            #db에 저장 시킬 명사화된 article title 하지만 넣을지 말지 고민....
            nouns_title = clean_text(str(spliter.nouns(title)))
            
            #마지막 페이지를 구별하기 위해 만듬
            if temp_page != page:
                first_url = url
                temp_page += 1
            """
            db에 저장시킬 함수 호출
            db_manager(db_id, url, title, str(yesterdayDate))
            print(url)
            print(title)
            print(nouns_title)
            """
            print(url)
            print(title)
            #날짜 폴더가 생성되어 있어야한다. 본문내용 함수 호출
            article_text(url, num, str(yesterdayDate))
            
            num += 1
            db_id += 1
        page += 1

In [51]:
if __name__ == '__main__':
    main()

http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=025&aid=0002742992
['홍준표, 류석춘ㆍ혁신위원들 격려…“잘된다고 생각하자”']
홍준표 류석 혁신 위원 격려
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0009459986
['한일 외교장관회담서 위안부 합의 입장차 재확인(종합2보)']
한일 외교 장관 회담 위안부 합의 입장 차 재 확인 종합 보
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=018&aid=0003892526
['강경화 장관, 日에 장관 직속 위안부 TF출범 의미 설명']
강경화 장관 장관 직속 위안부 출범 의미 설명
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0009459985
['北, 안보리 결의 환영 南정부 비난…"사대 매국 극치"']
안보리 결의 환영 정부 비난 사대 매국 극치
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=056&aid=0010490823
['한일 외교장관회담…강경화 “어려운 문제 소통하며 풀자”']
한일 외교 장관 회담 강경화 문제 소통
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=025&aid=0002742990
['강경화-고노 만났지만…위안부 합의 입장차 재확인']
강경화 위안부 합의 입장 차 재 확인
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=002&aid=0002036736
['朴지지자들이 법원서 장애인에 욕설을 퍼부은 이유']
지지자 법원 장애인 욕설 퍼부 이유
htt

http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=032&aid=0002808860
['[이재용 징역 12년 구형]박 특검 “변명에 반성 없다” 이재용 “국민연금 손해는 오해”']
이재용 징역 년 구형 박 특검 변명 반성 이재용 국민연금 손해 오해
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=032&aid=0002808859
['[이재용 징역 12년 구형]박근혜 독대 때 ‘부정 청탁’ 했는지, 정유라 알았는지가 핵심']
이재용 징역 년 구형 박근혜 독대 때 부정 청탁 정유 핵심
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=437&aid=0000160222
["[비하인드 뉴스] '새정치'에서 '새 사람'?…안철수 출마 '내홍'"]
비하인드 뉴스 새정치 에서 새 사람 안철수 출마 홍
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=437&aid=0000160221
["[팩트체크] '대통령 회고록' 출판 금지한 나라 없다?"]
팩트 체크 대통령 회고록 출판 금지 나라
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=015&aid=0003806894
['"갑질 문화 근절하라"']
문화 근절 하라
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=079&aid=0002997968
['"박찬주, 가전제품까지 가져가…갑질이 아니라 범죄"']
박찬주 가전제품 범죄
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=052&aid=0001043836
['문재인 대통령 "공관병 갑질 매우 유감...

http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0009459932
['북한 입장 전달하는 방광혁 부국장']
북한 입장 방광 혁 부국
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0009459931
['리용호 외무상, 회견 대신 연설문']
리용호 외무 회견 대신 연 설문
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=022&aid=0003198445
["56분·23분…한·미·일 '한반도 위기' 공조 방안 협의"]
분 분 일 한반도 위기 공조 방안 협의
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0009459930
['리용호 외무상, 회견 대신 연설문']
리용호 외무 회견 대신 연 설문
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=079&aid=0002997963
['文 "핵잠수함 보유 필요" …北 미사일에 강경 메시지']
핵잠수함 보유 필요 미사일 강경 메시지
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0002879655
['北 "핵 무력 강화에 양보 없을 것"']
핵 무력 강화 양보 것
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=437&aid=0000160216
["[앵커브리핑] 사병화에 길들여진…'장군은 뛰지 않는다'"]
앵커 브리핑 사병 길 여진 장군
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0002879

http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0009459914
['대화하는 강경화 장관-리용호 외무상']
강경화 장관 리용호 외무
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=448&aid=0000218586
['문 대통령 "갑질 뿌리뽑아야"…송영무 "불합리한 사적 지시 근절" ']
문 대통령 뿌리 송영 무 불합리 사적 지시 근절
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=015&aid=0003806846
['"핵에는 핵으로"…\'전술핵 재배치\' 꺼낸 한국당']
핵 핵 전술핵 재 배치 한국
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=448&aid=0000218587
['박찬주 대장 부인 "공관병, 아들처럼 대했는데"…들끓는 여론']
박찬주 대장 부인 공관 병 아들 여론
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=437&aid=0000160201
["북 이용호 외무상, 마닐라서 '핵개발 정당성' 옹호 주장"]
북 이용호 외무 마닐라 핵 개발 정당성 옹호 주장
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0009459913
['리용호 북 외무상과 대화하는 강경화 장관']
리용호 북 외무 강경화 장관
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=015&aid=0003806842
['"정기국회서 100대 개혁과제 추진에 총력"전열 재정비 나선 민주당']
정기 국회 개혁 과제 추진 총력 전열 재정 비 나선 민주당
http://n

http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=056&aid=0010490764
['北 리용호 “美 적대시정책 청산없이 핵·미사일 협상 없다”']
리용호 적대시 정책 청산 핵 미사일 협상
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=020&aid=0003085455
['하태경 “수능 절대평가 되면 금수저 전형 더욱 더 확대 될것”']
하태경 수능 절대평가 금수 저 전형 더욱 더 확대
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=214&aid=0000782505
["안철수 출마 강행…당내 만류 속 '결선투표' 새 변수"]
안철수 출마 강행 당내 만류 속 결선 투표 새 변수
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0002879612
['결선투표제 도입한 국민의당, 누가 가장 유리할까?']
결선투표제 도입 국민의당 누가 가장
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=437&aid=0000160192
['문 대통령-트럼프 통화…"북과 대화시도 해봤나" 물어와']
문 대통령 트럼프 통화 북 대화 시도 물어
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=025&aid=0002742970
['이용호 北 외무상 "미국의 적대시 정책 청산 없이 핵·미사일 협상 없다"']
이용호 외무 미국 적대시 정책 청산 핵 미사일 협상
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=018&aid=0003892483
['북한 리용호 "미국 적대정책 청산없이 핵 협상 없다

...

http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0009457755
['도종환 장관 "촛불집회의 힘 평창올림픽으로 잇고 싶다"']
도종환 장관 촛불집회 힘 평창올림픽
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=056&aid=0010490445
['中 “北, 핵·미사일 추가 도발 말라”…ARF 오늘 개막']
핵 미사일 추가 도발 오늘 개막
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=056&aid=0010490443
['한미일 “만장일치 높이 평가”…北 “美 불바다”']
한미 일 만장일치 높이 평가 불 바다
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=056&aid=0010490442
['“병사에게 농사도 시켜”…박찬주 사령관 부인 오늘 소환']
병사 농사 박찬주 사령관 부인 오늘 소환
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=016&aid=0001273972
['공관병 갑질 후폭풍…별들이 떨고 있다']
공관 병 후 폭풍 별
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=056&aid=0010490433
['中 “北, 핵·미사일 추가 도발 말라”…ARF 오늘 개막']
핵 미사일 추가 도발 오늘 개막
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=056&aid=0010490429
['국민의당 내홍 격화…당권 주자 여론전']
국민의당 홍 격화 당권 주자 여론 전
http://news.naver.com/main/read.nhn?mode=LSD&mid=

http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=015&aid=0003806350
['육사 경쟁률 33 대 1 최고 "취업난에 군장교 선호"']
육사 경쟁률 최고 취업 난 군 장교 선호
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=214&aid=0000782341
["'원유 공급 중단' 제외…北 태도변화 효과는?"]
원유 공급 중단 제외 태도 변화 효과
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=018&aid=0003891646
['[팩트체크]중국은 왜 북한을 비호할까']
팩트 체크 중국 왜 북한 비호
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=214&aid=0000782339
["ARF 외교장관회의…'대북 대응·사드 문제' 논의"]
외교 장관 회의 대북 대응 사드 문제 논의
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=079&aid=0002997535
['"구태·몰상식"…격해지는 安 반발기류, 국민의당 파국속으로']
구태 몰상식 반발 기류 국민의당 파국 속
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=032&aid=0002808597
['[단독]“국정원 위법사항 당연히 검찰에 수사 요청…이명박의 정치개입 지시·묵인 여부 밝혀야”']
단독 국정원 위법 사항 검찰 수사 요청 이명박 정치 개입 지시 묵인 여부
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=079&aid=0002997531
['문 대통령 집무실 책상에 쌓인 4대 과제 ']
문 대통령 집무실 책상 

http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=008&aid=0003915546
['담뱃값 올릴땐 "국민건강", 정권 뺏기니 "서민증세"']
담뱃값 땐 국민 건강 정권 서민 증세
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=008&aid=0003915543
['담뱃값 인하? 국제협약 위반 "국제적 망신"']
담뱃값 인하 국제 협약 위반 국제 망신
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0002877868
['이재용 재판 오늘 결심…특검 구형 형량 주목']
이재용 재판 오늘 결심 특검 구형 형량 주목
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0009457650
['오늘 마닐라서 ARF 개막…北에 도발중단 요구 거셀듯']
오늘 마닐라 개막 도발 중단 요구 거
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=052&aid=0001043429
['안철수 "독배 마시는 심정으로 당권 도전"']
안철수 독배 심정 당권 도전
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=081&aid=0002843348
['[역사 속 북소리] 엄동설한에 거리 나선 소녀의 사연']
역사 속 북 소리 엄동설한 거리 나선 소녀 사연
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=081&aid=0002843341
['경력개방직, 혁신과 역차별 사이']
경력 방직 혁신 역차별 사이
http://news.naver.com/main/read.nhn?mode=LSD&mi

http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=025&aid=0002742693
['[현장에서] 제재 8시간 뒤 이용호 만난 왕이 “더 이상 핵실험, 미사일 발사 말라”']
현장 제재 시간 뒤 이용호 만난 왕 더 이상 핵실험 미사일 발사
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=025&aid=0002742691
['현안 쌓였는데 … 주미 대사, 정부 출범 석달째 결정 못해']
현안 주 대사 정부 출범 석달 결정 못
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=005&aid=0001015603
['“2019년에 한반도 핵전쟁 난다” 영국 언론이 예상한 시나리오']
년 한반도 핵전쟁 난 영국 언론 시나리오
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=025&aid=0002742687
['퇴근 후 카톡금지 법안 발의 “계속 업무 지시 땐 연장수당 줘야”']
퇴근 후 카톡 금지 법안 발의 계속 업무 지시 땐 연장 수당
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=025&aid=0002742686
['‘전기 충격’ 필요하다는 안철수 … 천정배 “구태 정치” 정동영 “사당화 하나”']
전기 충격 안철수 천정배 구태 정치 정동영 사당 하나
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=025&aid=0002742685
['[단독] 갑질 지휘관은 진급 안 시킨다 … ‘별 물갈이’ 빨라질 듯']
단독 지휘 관 진급 안 별 물 갈이 듯
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&

http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0009457595
['리용호 북한 외무상 ARF 환영만찬 참석']
리용호 북한 외무 환영 만찬 참석
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0008107424
['[종합]국민의당 의원들 "安, 전대 출마 만류키로...7일 오후 5시 면담"']
종합 국민의당 의원 전대 출마 만류 일 오후 시 면담
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=005&aid=0001015592
['이재용 오늘 ‘세기의 재판’…  재산국외도피·횡령 혐의 막판 변수']
이재용 오늘 세기 재판 재산 국외 도피 횡령 혐의 막판 변수
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0009457594
['[인사] 국회']
인사 국회
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0002877858
['[인사] 국회']
인사 국회
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0002877857
['[인사] 국회도서관']
인사 국회 도서관
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=298&aid=0000225677
['도심 ‘개방 휴식공간’ 내 무단영업 등에 벌금 부과 추진']
도심 개방 휴식 공간 내 무단 영업 등 벌금 부과 추진
